### CS 109A/STAT 121A/AC 209A/CSCI E-109A: Homework 1
Harvard University
Fall 2017
Instructors: Pavlos Protopapas, Kevin Rader, Rahul Dave, Margo Levine

### INSTRUCTIONS
WARNING: There is web page scraping in this homework. It takes about 40 minutes. Do not wait till the last minute to do this homework.

To submit your assignment follow the instructions given in canvas.
Restart the kernel and run the whole notebook again before you submit. There is an important CAVEAT to this. DO NOT run the web-page fetching cells again. (We have provided hints like # DO NOT RERUN THIS CELL WHEN SUBMITTING on some of the cells where we provide the code). Instead load your data structures from the JSON files we will ask you to save below. Otherwise you will be waiting for a long time. (Another reason to not wait until the last moment to submit.)

Do not include your name in the notebook.



### Homework 1: Rihanna or Mariah?
Billboard Magazine puts out a top 100 list of "singles" every week. Information from this list, as well as that from music sales, radio, and other sources is used to determine a top-100 "singles" of the year list. A single is typically one song, but sometimes can be two songs which are on one "single" record.

In this homework you will:

Scrape Wikipedia to obtain infprmation about the best singers and groups from each year (distinguishing between the two groups) as determined by the Billboard top 100 charts. You will have to clean this data. Along the way you will learn how to save data in json files to avoid repeated scraping.
Scrape Wikipedia to obtain information on these singers. You will have to scrape the web pages, this time using a cache to guard against network timeouts (or your laptop going to sleep). You will again clean the data, and save it to a json file.
Use pandas to represent these two datasets and merge them.
Use the individual and merged datasets to visualize the performance of the artists and their songs. We have kept the amount of analysis limited here for reasons of time; but you might enjoy exploring music genres and other aspects of the music business you can find on these wikipedia pages at your own leisure.
You should have worked through Lab0 and Lab 1, and Lecture 2. Lab 2 will help as well.

As usual, first we import the necessary libraries. In particular, we use Seaborn to give us a nicer default color palette, with our plots being of large (poster) size and with a white-grid background.

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

### Q1. Scraping Wikipedia for Billboard Top 100.
In this question you will scrape Wikipedia for the Billboard's top 100 singles.

#### Scraping Wikipedia for Billboard singles
We'll be using BeautifulSoup, and suggest that you use Python's built in requests library to fetch the web page

### 1.1 Parsing the Billboard Wikipedia page for 1970
Obtain the web page at http://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_1970 using a HTTP GET request. From this web page we'll extract the top 100 singles and their rankings. Create a list of dictionaries, 100 of them to be precise, with entries like

{'url': '/wiki/Sugarloaf_(band)', 'ranking': 30, 'band_singer': 'Sugarloaf', 'title': 'Green-Eyed Lady'}.

If you look at that web page, you'll see a link for every song, from which you can get the url of the singer or band. We will use these links later to scrape information about the singer or band. From the listing we can also get the band or singer name band_singer, and title of the song.

HINT: look for a table with class wikitable.

You should get something similar to this (where songs is the aforementioned list):

songs[2:4]
[{'band_singer': 'The Guess Who',
  'ranking': 3,
  'title': '"American Woman"',
  'url': '/wiki/The_Guess_Who'},
 {'band_singer': 'B.J. Thomas',
  'ranking': 4,
  'title': '"Raindrops Keep Fallin\' on My Head"',
  'url': '/wiki/B.J._Thomas'}]

In [2]:
import requests

data = requests.get("https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_1970")

In [3]:
from bs4 import BeautifulSoup
page = data.text
soup = BeautifulSoup(page, 'html.parser')
soup.table["class"]

table_html = soup.find('table', attrs={'class':'wikitable sortable'})

In [4]:
songs_list = []

table_row_list = table_html.find_all('tr')
table_rows = table_row_list[1:] 
table_rows

for row in table_rows:
#     print(row)
    table_row_values = row.find_all('td')
    
#     print(table_cols)
    
    ranking = table_row_values[0].get_text()
    title = table_row_values[1].get_text()
    band_singer = table_row_values[2].get_text().replace("\n","")
    
    url = table_row_values[1].a["href"]
    
    dic = {'band_singer': band_singer,
           'ranking': ranking,
           'title': title,
           'url': url}
    songs_list.append(dic)
print(songs_list[2:4])

[{'band_singer': 'The Guess Who', 'ranking': '3', 'title': '"American Woman"', 'url': '/wiki/American_Woman'}, {'band_singer': 'B.J. Thomas', 'ranking': '4', 'title': '"Raindrops Keep Fallin\' on My Head"', 'url': '/wiki/Raindrops_Keep_Fallin%27_on_My_Head'}]


### 1.2 Generalize the previous: scrape Wikipedia from 1992 to 2014
By visiting the urls similar to the ones for 1970, we can obtain the billboard top 100 for the years 1992 to 2014. (We choose these later years rather than 1970 as you might find music from this era more interesting.) Download these using Python's requests module and store the text from those requests in a dictionary called yearstext. This dictionary ought to have as its keys the years (as integers from 1992 to 2014), and as values corresponding to these keys the text of the page being fetched.

You ought to sleep a second (look up time.sleep in Python) at the very least in-between fetching each web page: you do not want Wikipedia to think you are a marauding bot attempting to mount a denial-of-service attack.

HINT: you might find range and string-interpolation useful to construct the URLs .

In [5]:
years = {}

for y in range(1992,2015):
    st = "https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_"+ str(y)
    temp_req = requests.get(st)
    years[y] = temp_req.text

### 1.3 Parse and Clean data
Remember the code you wrote to get data from 1970 which produces a list of dictionaries, one corresponding to each single. Now write a function parse_year(the_year, yeartext_dict) which takes the year, prints it out, gets the text for the year from the just created yearstext dictionary, and return a list of dictionaries for that year, with one dictionary for each single. Store this list in the variable yearinfo.

The dictionaries must be of this form:

{'band_singer': ['Brandy', 'Monica'],
  'ranking': 2,
  'song': ['The Boy Is Mine'],
  'songurl': ['/wiki/The_Boy_Is_Mine_(song)'],
  'titletext': '" The Boy Is Mine "',
  'url': ['/wiki/Brandy_Norwood', '/wiki/Monica_(entertainer)']}
The spec of this function is provided below:

In [6]:
"""
Function
--------
parse_year

Inputs
------
the_year: the year you want the singles for
yeartext_dict: a dictionary with keys as integer years and values the downloaded web pages 
    from wikipedia for that year.
   
Returns
-------

a list of dictionaries, each of which corresponds to a single and has the
following data:

Eg:

{'band_singer': ['Brandy', 'Monica'],
  'ranking': 2,
  'song': ['The Boy Is Mine'],
  'songurl': ['/wiki/The_Boy_Is_Mine_(song)'],
  'titletext': '" The Boy Is Mine "',
  'url': ['/wiki/Brandy_Norwood', '/wiki/Monica_(entertainer)']}
  
A dictionary with the following data:
    band_singer: a list of bands/singers who made this single
    song: a list of the titles of songs on this single
    songurl: a list of the same size as song which has urls for the songs on the single 
        (see point 3 above)
    ranking: ranking of the single
    titletext: the contents of the table cell
    band_singer: a list of bands or singers on this single
    url: a list of wikipedia singer/band urls on this single: only put in the part 
        of the url from /wiki onwards
    

Notes
-----
See description and example above.
"""
def parse_year(the_year, yeartext_dict):
    year = the_year
    yearinfo = []
    song = []
    songurl = []
    band_singer = []
    title = []
    url = []
    title_text = ''
    i = 0
    title_string = ''
    band_singer = ''
    soup = BeautifulSoup(years[year], "lxml")
    tables = soup.find('table', attrs={'class':'wikitable sortable'})        
    #iterates through tree structure, scraping our data
    tr_list = tables.find_all('tr')    
    for tr in tr_list:
        td_list = tr.find_all('td')
        if td_list == [] :
            td_list = []
        else : 
            ranking = tr.th.string
            links = tr.td.findAll('a')
            number_of_links = len(links)   
            if number_of_links == 0:
                songurl = [None]
                title_text = [a['title']]
                song.append(a['title'] )
            else :
                i = 0
                for a in tr.td.findAll('a') : 
                    song.append(a['title'] )
                    title_string = '\"' + a['title'] + '\"'    
                    if i == 0 :
                        title_text = title_string
                        i = i + 1
                    else :
                        title_text = title_text + ' / ' + title_string
                        i = i + 1    
                    songurl.append(a['href'])
            title = song
            #finds next td tag
            tr.td.findNext('td') 
            temp = len(tr.td.findNext('td').findAll('a'))
            if temp == 0:
                singer_url = [None]
                band_singer = tr.td.findNext('td').string
                band_singer = [band_singer]
            elif temp == 1:
                singer_url = tr.td.findNext('td').a['href']
                singer_url = [singer_url]
                band_singer = tr.td.findNext('td').a.string
                band_singer = [band_singer]
            else:
                singer_url = []
                band_singer = []
                for a in tr.td.findNext('td').findAll('a'):
                    webpage = a['href']
                    singer_url.append(webpage)
                    band_singer.append(a.string)            
            #creates dictionary entry                   
            dict_entry = {'band_singer' : band_singer,
            'ranking' : ranking.replace("\n",""),
            'song' : title, 'songurl': songurl, 'titletext' : title_text,
            'url' : singer_url}
            #appends new dictionary to our list
            yearinfo.append(dict_entry)      
            songurl = []
            song = []
            title_string = ''
            title_text = ''    
    return(yearinfo)

In [7]:
years_info = {}
for y in range(1992,2005):
  years_info.update({y: parse_year(y,years)})

In [8]:
print(years_info[1997])

[{'band_singer': ['Elton John'], 'ranking': '1', 'song': ['Something About the Way You Look Tonight', 'Candle in the Wind 1997'], 'songurl': ['/wiki/Something_About_the_Way_You_Look_Tonight', '/wiki/Candle_in_the_Wind_1997'], 'titletext': '"Something About the Way You Look Tonight" / "Candle in the Wind 1997"', 'url': ['/wiki/Elton_John']}, {'band_singer': ['Jewel'], 'ranking': '2', 'song': ['Foolish Games', 'You Were Meant for Me (Jewel song)'], 'songurl': ['/wiki/Foolish_Games', '/wiki/You_Were_Meant_for_Me_(Jewel_song)'], 'titletext': '"Foolish Games" / "You Were Meant for Me (Jewel song)"', 'url': ['/wiki/Jewel_(singer)']}, {'band_singer': ['Puff Daddy', 'Faith Evans', '112'], 'ranking': '3', 'song': ["I'll Be Missing You"], 'songurl': ['/wiki/I%27ll_Be_Missing_You'], 'titletext': '"I\'ll Be Missing You"', 'url': ['/wiki/Sean_Combs', '/wiki/Faith_Evans', '/wiki/112_(band)']}, {'band_singer': ['Toni Braxton'], 'ranking': '4', 'song': ['Un-Break My Heart'], 'songurl': ['/wiki/Un-Brea

#### Helpful notes
Notice that some singles might have multiple songs:

{'band_singer': ['Jewel'],
  'ranking': 2,
  'song': ['Foolish Games', 'You Were Meant for Me'],
  'songurl': ['/wiki/Foolish_Games',
   '/wiki/You_Were_Meant_for_Me_(Jewel_song)'],
  'titletext': '" Foolish Games " / " You Were Meant for Me "',
  'url': ['/wiki/Jewel_(singer)']}
And some singles don't have a song URL:

{'band_singer': [u'Nu Flavor'],
  'ranking': 91,
  'song': [u'Heaven'],
  'songurl': [None],
  'titletext': u'"Heaven"',
  'url': [u'/wiki/Nu_Flavor']}
Thus there are some issues this function must handle:

There can be more than one  band_singer as can be seen above (sometimes with a comma, sometimes with "featuring" in between). The best way to parse these is to look for the urls.
There can be two songs in a single, because of the way the industry works: there are two-sided singles. See https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_1997 for an example. You can find other examples in 1998 and 1999.
The titletext is the contents of the table cell, and retains the quotes that Wikipedia puts on the single.
If no song anchor is found (see the 24th song in the above url), assume there is one song in the single, set songurl to [None] and the song name to the contents of the table cell with the quotes stripped (ie song is a one-element list with this the  titletext stripped of its quotes).
As a check, we can do this for 1997. We'll print the first 5 outputs: parse_year(1997, yearstext)[:5]

This should give the following. Notice that the year 1997 exercises the edge cases we talked about earlier.

[{'band_singer': ['Elton John'],
  'ranking': 1,
  'song': ['Something About the Way You Look Tonight',
   'Candle in the Wind 1997'],
  'songurl': ['/wiki/Something_About_the_Way_You_Look_Tonight',
   '/wiki/Candle_in_the_Wind_1997'],
  'titletext': '" Something About the Way You Look Tonight " / " Candle in the Wind 1997 "',
  'url': ['/wiki/Elton_John']},
 {'band_singer': ['Jewel'],
  'ranking': 2,
  'song': ['Foolish Games', 'You Were Meant for Me'],
  'songurl': ['/wiki/Foolish_Games',
   '/wiki/You_Were_Meant_for_Me_(Jewel_song)'],
  'titletext': '" Foolish Games " / " You Were Meant for Me "',
  'url': ['/wiki/Jewel_(singer)']},
 {'band_singer': ['Puff Daddy', 'Faith Evans', '112'],
  'ranking': 3,
  'song': ["I'll Be Missing You"],
  'songurl': ['/wiki/I%27ll_Be_Missing_You'],
  'titletext': '" I\'ll Be Missing You "',
  'url': ['/wiki/Sean_Combs', '/wiki/Faith_Evans', '/wiki/112_(band)']},
 {'band_singer': ['Toni Braxton'],
  'ranking': 4,
  'song': ['Un-Break My Heart'],
  'songurl': ['/wiki/Un-Break_My_Heart'],
  'titletext': '" Un-Break My Heart "',
  'url': ['/wiki/Toni_Braxton']},
 {'band_singer': ['Puff Daddy', 'Mase'],
  'ranking': 5,
  'song': ["Can't Nobody Hold Me Down"],
  'songurl': ['/wiki/Can%27t_Nobody_Hold_Me_Down'],
  'titletext': '" Can\'t Nobody Hold Me Down "',
  'url': ['/wiki/Sean_Combs', '/wiki/Mase']}]


### Save a json file of information from the scraped files
We do not want to lose all this work, so let's save the last data structure we created to disk. That way if you need to re-run from here, you don't need to redo all these requests and parsing.

DO NOT RERUN THE HTTP REQUESTS TO WIKIPEDIA WHEN SUBMITTING.

We DO NOT need to see these JSON files in your submission!

In [9]:
import json

Now let's reload our JSON file into the yearinfo variable, just to be sure everything is working.


In [10]:
# RERUN WHEN SUBMITTING
# Another way to deal with files. Has the advantage of closing the file for you.
with open("yearinfo.json", "r") as fd:
    yearinfo = json.load(fd)

### 1.4 Construct a year-song-singer dataframe from the yearly information
Let's construct a dataframe flatframe from the yearinfo. The frame should be similar to the frame below. Each row of the frame represents a song, and carries with it the chief properties of year, song, singer, and ranking.


![alt text](https://camo.githubusercontent.com/f5b8fd1a5791d836bc4e1f7b546d4f6f62d97bf7/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f63733130392f612d323031372f6d61737465722f68776173736574732f696d616765732f4857315343312e706e67 "Title")

In [11]:
rows = []
for year in yearinfo.keys():
    for song in yearinfo[year]:
        song['year'] = year
        rows.append(song)

In [12]:
band_singer = []
songurl = ''
title_text = ''
singer_url = []
starting_length = len(rows)
for dics in rows: 
    if starting_length == 0:
        break
    dict_add = {}   
    # checks if our dictionary contains lists longer than one element
    if len(dics['band_singer']) > 1:
        i = 0
        j = len(dics['band_singer'])
        
        for value in dics['band_singer']:
            # new dictionary entry to append to our list
            dict_add = {'band_singer' : dics['band_singer'][i],
            'ranking' : dics['ranking'],
            'song' : dics['song'], 'songurl': dics['songurl'], 'titletext' : dics['titletext'],
            'url' : dics['url'][i], 'year' : dics['year']}
            rows.append(dict_add)
            i = 1 + i
            j = j - 1
    starting_length = starting_length - 1

rows2 = []
band_singer = []

# loops through our list again, removing duplicate entries
for dics in rows:
    if len(dics['band_singer']) == 1 or len(dics['band_singer']) > 5:
        rows2.append(dics)

# turns all remaining one element lists into strings
for row in rows2:
    for key in row:
        row[key] = str(row[key])
        row[key] = row[key].strip("[]")
        row[key] = row[key].strip("''")

In [13]:
rows2
flatframe = pd.DataFrame(rows2)


# check datatypes of dataframe columns
flatframe['year'].dtype
flatframe

,band_singer,ranking,song,songurl,titletext,url,year
0,Boyz II Men,1\n,End of the Road,/wiki/End_of_the_Road,"""End of the Road""",/wiki/Boyz_II_Men,1992
1,Sir Mix-a-Lot,2\n,Baby Got Back,/wiki/Baby_Got_Back,"""Baby Got Back""",/wiki/Sir_Mix-a-Lot,1992
2,Kris Kross,3\n,Jump (Kris Kross song),/wiki/Jump_(Kris_Kross_song),"""Jump (Kris Kross song)""",/wiki/Kris_Kross,1992
3,Vanessa Williams,4\n,Save the Best for Last,/wiki/Save_the_Best_for_Last,"""Save the Best for Last""",/wiki/Vanessa_L._Williams,1992
4,TLC,5\n,Baby-Baby-Baby,/wiki/Baby-Baby-Baby,"""Baby-Baby-Baby""",/wiki/TLC_(band),1992
5,Eric Clapton,6\n,Tears in Heaven,/wiki/Tears_in_Heaven,"""Tears in Heaven""",/wiki/Eric_Clapton,1992
6,En Vogue,7\n,"""My Lovin' (You're Never Gonna Get It)""",/wiki/My_Lovin%27_(You%27re_Never_Gonna_Get_It),"""My Lovin' (You're Never Gonna Get It)""",/wiki/En_Vogue,1992
7,Red Hot Chili Peppers,8\n,Under the Bridge,/wiki/Under_the_Bridge,"""Under the Bridge""",/wiki/Red_Hot_Chili_Peppers,1992
8,Color Me Badd,9\n,All 4 Love,/wiki/All_4_Love,"""All 4 Love""",/wiki/Color_Me_Badd,1992
9,Jon Secada,10\n,Just Another Day (Jon Secada song),/wiki/Just_Another_Day_(Jon_Secada_song),"""Just Another Day (Jon Secada song)""",/wiki/Jon_Secada,1992


To construct the dataframe, we'll need to iterate over the years and the singles per year. Notice how, above, the dataframe is ordered by ranking and then year. While the exact order is up to you, note that you will have to come up with a scheme to order the information.

Check that the dataframe has sensible data types. You will also likely find that the year field has become an "object" (Pandas treats strings as generic objects): this is due to the conversion to and back from JSON. Such conversions need special care. Fix any data type issues with flatframe. (See Pandas astype function.) We will use this flatframe in the next question.

(As an aside, we used the name flatframe to indicate that this dataframe is flattened from a hierarchical dictionary structure with the keys being the years.)

Who are the highest quality singers?
Here we show the highest quality singers and plot them on a bar chart.

### 1.5 Find highest quality singers according to how prolific they are
What do we mean by highest quality? This is of course open to interpretation, but let's define "highest quality" here as the number of times a singer appears in the top 100 over this time period. If a singer appears twice in a year (for different songs), this is counted as two appearances, not one.

Make a bar-plot of the most prolific singers. Singers on this chart should have appeared at-least more than 15 times. (HINT: look at the docs for the pandas method value_counts.)

### 1.6 What if we used a different metric?

In [14]:
artist_count = flatframe["band_singer"].value_counts()

artist_count

Rihanna                       35
Lil Wayne                     31
Ludacris                      29
R. Kelly                      28
Mariah Carey                  27
                              ..
O-Town                         1
Darius Rucker                  1
Beenie Man                     1
Mint Condition                 1
Lisa Loeb and Nine Stories     1
Name: band_singer, Length: 999, dtype: int64

What we would like to capture is this: a singer should to be scored higher if the singer appears higher in the rankings. So we'd say that a singer who appeared once at a higher and once at a lower ranking is a "higher quality" singer than one who appeared twice at a lower ranking.

To do this, group all of a singers songs together and assign each song a score 101 - ranking. Order the singers by their total score and make a bar chart for the top 20.

### 1.7 Do you notice any major differences when you change the metric?¶
How have the singers at the top shifted places? Why do you think this happens?

### Q2. Scraping and Constructing: Information about Artists, Bands and Genres from Wikipedia
Our next job is to use those band/singer urls we collected under flatframe.url and get information about singers and/or bands.

#### Scrape information about artists from wikipedia
We wish to fetch information about the singers or groups for all the winning songs in a list of years.

Here we show a function that fetches information about a singer or group from their url on wikipedia. We create a cache object urlcache that will avoid redundant HTTP requests (e.g. an artist might have multiple singles on a single year, or be on the list over a span of years). Once we have fetched information about an artist, we don't need to do it again. The caching also helps if the network goes down, or the target website is having some problems. You simply need to run the get_page function below again, and the urlcache dictionary will continue to be filled.

If the request gets an HTTP return code different from 200, (such as a 404 not found or 500 Internal Server Error) the cells for that URL will have a value of 1; and if the request completely fails (e.g. no network connection) the cell will have a value of 2. This will allow you to analyse the failed requests.

Notice that we have wrapped the call in whats called an exception block. We try to make the request. If it fails entirely, or returns a HTTP code thats not 200, we set the status to 2 and 1 respectively.

In [15]:
urlcache={}

In [16]:
def get_page(url):
    # Check if URL has already been visited.
    if (url not in urlcache) or (urlcache[url]==1) or (urlcache[url]==2):
        time.sleep(1)
        # try/except blocks are used whenever the code could generate an exception (e.g. division by zero).
        # In this case we don't know if the page really exists, or even if it does, if we'll be able to reach it.
        try:
            r = requests.get("http://en.wikipedia.org%s" % url)

            if r.status_code == 200:
                urlcache[url] = r.text
            else:
                urlcache[url] = 1
        except:
            urlcache[url] = 2
    return urlcache[url]

We sort the flatframe by year, ascending, first. Think why.

In [17]:
flatframe=flatframe.sort_values('year')
flatframe.head()
urlcache

{}

##### Pulling and saving the data
You may have to run this function again and again, in case there were network problems. Note that, because there is a "global" cache, it will take less time each time you run it. Also note that this function is designed to be run again and again: it attempts to make sure that there are no unresolved pages remaining. Let us make sure of this: the sum below should be 0, and the boolean True.

In [18]:
# DO NOT RERUN THIS CELL WHEN SUBMITTING
# Here we are populating the url cache
# subsequent calls to this cell should be very fast, since Python won't
# need to fetch the page from the web server.
# NOTE this function will take quite some time to run (about 30 mins for me), since we sleep 1 second before
# making a request. If you run it again it will be almost instantaneous, save requests that might have failed
# (you will need to run it again if requests fail..see cell below for how to test this)
flatframe["url"].apply(get_page)

0       <!DOCTYPE html>\n<html class="client-nojs" lan...
78      <!DOCTYPE html>\n<html class="client-nojs" lan...
77      <!DOCTYPE html>\n<html class="client-nojs" lan...
76      <!DOCTYPE html>\n<html class="client-nojs" lan...
74      <!DOCTYPE html>\n<html class="client-nojs" lan...
                              ...                        
1698    <!DOCTYPE html>\n<html class="client-nojs" lan...
1697    <!DOCTYPE html>\n<html class="client-nojs" lan...
1696    <!DOCTYPE html>\n<html class="client-nojs" lan...
1702    <!DOCTYPE html>\n<html class="client-nojs" lan...
2749    <!DOCTYPE html>\n<html class="client-nojs" lan...
Name: url, Length: 2750, dtype: object

You may have to run this function again and again, in case there were network problems. Note that, because there is a "global" cache, it will take less time each time you run it. Also note that this function is designed to be run again and again: it attempts to make sure that there are no unresolved pages remaining. Let us make sure of this: the sum below should be 0, and the boolean True.

In [19]:
# DO NOT RERUN THIS CELL WHEN SUBMITTING
print("Number of bad requests:",np.sum([(urlcache[k]==1) or (urlcache[k]==2) for k in urlcache])) # no one or 0's)
print("Did we get all urls?", len(flatframe.url.unique())==len(urlcache)) # we got all of the urls

Number of bad requests: 11
Did we get all urls? True


Let's save the urlcache to disk, just in case we need it again.

In [20]:
# DO NOT RERUN THIS CELL WHEN SUBMITTING
with open("data/artistinfo.json","w") as fd:
    json.dump(urlcache, fd)
del urlcache

FileNotFoundError: [Errno 2] No such file or directory: 'data/artistinfo.json'

In [ ]:
# RERUN WHEN SUBMITTING
with open("artistinfo.json") as json_file:
    urlcache = json.load(json_file)
    
urlcache

### 2.1 Extract information about singers and bands
From each page we collected about a singer or a band, extract the following information:

If the page has the text "Born" in the sidebar on the right, extract the element with the class .bday. If the page doesn't contain "Born", store False. Store either of these into the variable born. We want to analyze the artist's age.

If the text "Years active" is found, but no "born", assume a band. Store into the variable ya the value of the next table cell corresponding to this, or False if the text is not found.

Put this all into a function singer_band_info which takes the singer/band url as argument and returns a dictionary dict(url=url, born=born, ya=ya).

The information can be found on the sidebar on each such wikipedia page, as the example here shows:

![alt text](https://camo.githubusercontent.com/38f07e6ac1cd6d19afc3f197bb7e1267af8c14d9/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f63733130392f612d323031372f6d61737465722f68776173736574732f696d616765732f73616e64672e706e67 "Title")

Write the function singer_band_info according to the following specification:


In [ ]:
"""
Function
--------
singer_band_info

Inputs
------
url: the url
page_text: the text associated with the url
   
Returns
-------
A dictionary with the following data:
    url: copy the input argument url into this value
    born: the artist's birthday
    ya: years active variable

Notes
-----
See description above. Also note that some of the genres urls might require a 
bit of care and special handling.
"""


def singer_band_info(url, page_text):
    bday_dict = {}
    bday = ''
    ya = ''
    # soupify our webpage
    soup = BeautifulSoup(page_text[url], "lxml")
    tables = soup.find('table', attrs={'class':'infobox vcard plainlist'})
    if (tables == None):
        tables = soup.find('table', attrs={'class':'infobox biography vcard'})
    bday = tables.find('span', {'class': 'bday'})
    if bday: 
        bday = bday.get_text()[:4]
        bday_dict = {'url' : url, 'born' : bday, 'ya' : ya}
    else:
        ya = False
        for tr in tables.find_all('tr'):
            if hasattr(tr.th, 'span'):
                if hasattr(tr.th.span, 'string'):
                    if tr.th.span.string == 'Years active':                
                        if hasattr(tr.th, 'span'):
                            #ya = tr.td.string
                            ya = tr.td.text   #DK add
                            bday = 'False'
                            bday_dict = {'url' : url, 'born' : 'False', 'ya' : ya}
    return(bday_dict)

In [ ]:
url = '/wiki/Iggy_Azalea'
singer_band_info(url, urlcache)

#### 2.2 Merging this information in
Iterate over the items in the singer-group dictionary cache urlcache, run the above function, and create a dataframe from there with columns url, born, and ya. Merge this dataframe on the url key with flatframe, creating a rather wide dataframe that we shall call largedf. It should look something like this:

![alt text](https://camo.githubusercontent.com/530220965d9e56fb1366f3eccf32fccf12384dff/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f63733130392f612d323031372f6d61737465722f68776173736574732f696d616765732f4857315343332e706e67 "Title")

Notice how the born and ya and url are repeated every time a different song from a given band is represented in a row.

In [ ]:
list_of_addit_dicts = []
bday_dict = {}
for url in urlcache.keys():   
    try:
        bday_dict = singer_band_info(url, urlcache)
        list_of_addit_dicts.append(bday_dict)
    except:
        break

In [ ]:
additional_df = pd.DataFrame(list_of_addit_dicts)

largedf = pd.merge(flatframe, additional_df, left_on='url', right_on='url', how="outer")
largedf